In [0]:
dict_cards = {
    '00357': {'bank': 'Bradesco', 'bandeira': 'American Express', 'short_descricao': 'Bradesco 00357', 'descricao': 'The Platinum Card Prime', 'full_descricao': 'The Platinum Card Prime 00357', 'is_current': True, 'dia_vencimento': 10}, 
    '0768': {'bank': 'Bradesco', 'bandeira': 'Mastercard', 'short_descricao': 'Bradesco 0768', 'descricao': 'Black Prime', 'full_descricao': 'Black Prime 0768', 'is_current': True, 'dia_vencimento': 5},
    '0840': {'bank': 'BTG Pactual', 'bandeira': 'Mastercard', 'short_descricao': 'BTG 0768', 'descricao': 'BTG Black', 'full_descricao': 'BTG Black 0840', 'is_current': True, 'dia_vencimento': 5},
    '0194': {'bank': 'BTG Pactual', 'bandeira': 'Mastercard', 'short_descricao': 'BTG 0194', 'descricao': 'BTG Black', 'full_descricao': 'BTG Black 0194', 'is_current': False, 'dia_vencimento': 5},
    '1399': {'bank': 'BTG Pactual', 'bandeira': 'Mastercard', 'short_descricao': 'BTG 1399', 'descricao': 'BTG Black', 'full_descricao': 'BTG Virtual Black 1399', 'is_current': False, 'dia_vencimento': 5},
}

In [0]:
data = [(chave, valor) for chave, valor in dict_cards.items()]

df_cards = spark.createDataFrame(data, ["card_final", "value"])

In [0]:
from pyspark.sql.functions import col

In [0]:
df_cards = df_cards.withColumn('bandeira', col('value.bandeira')) \
       .withColumn('descricao', col('value.descricao')) \
       .withColumn('bank', col('value.bank')) \
       .withColumn('full_descricao', col('value.full_descricao')) \
       .withColumn('is_current', col('value.is_current')) \
       .withColumn('dia_vencimento', col('value.dia_vencimento'))

In [0]:
df_cards = df_cards.drop('value')

In [0]:
display(df_cards)

As dimensões devem ser armazenadas na camada Silver, já com a lógica de SCD aplicada

🔹 SCD Tipo 1 – Sobrescrita (sem histórico)
- Lógica: substitui o valor antigo pelo novo, sem guardar histórico.
- 👉 Só existe a versão mais recente. Você não sabe que antes o cartão tinha outro nome.

🔹 SCD Tipo 2 – Histórico Completo
- Lógica: cria uma nova linha a cada mudança, com datas de vigência e flag de ativo.
- 👉 Aqui você sabe exatamente quando o cartão mudou de nome e quando foi cancelado.

🔹 SCD Tipo 3 – Histórico Parcial
- Lógica: mantém apenas o valor atual e o anterior em colunas adicionais.
- 👉 Você só consegue ver o valor atual e o imediatamente anterior, mas não todo o histórico.

✅ Conclusão
- SCD Tipo 1 → simples, mas sem histórico.
- SCD Tipo 2 → mais completo, preserva todo o ciclo de vida (ideal para cartões).
- SCD Tipo 3 → útil se só interessa comparar “antes e depois”, mas não múltiplas mudanças

In [0]:
df_cards.write \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .format('delta') \
    .saveAsTable('personalfinance.silver.dim_cards')